In [1]:
# IT IS LIGHT NORMALAZING! IT IS WITHOUT NTLK,TEXTBLOOB OR SPACY
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from time import time 
import string
from collections import Counter

num_words=300 #How many words we will use for modeling

#STOP_WORDS.Here is minus_words. This is list of words which can't help us in creating our model 
#Some ntlk stop words + my words. All words are without " ' ".
stop_words=['inside','ever','instead','already','theres','anything','thats','always','could','hers', 'only', 'neednt', 'doesnt', 'without', 'have', 'its', 'don', 'ive', 'into', 'before', 'mustn', 'any', 'now', 'such', 'you', 'trump', 'below', 'since', 'which', 'doesn', 'hasn', 'itself', 'all', 'someone', 'couldn', 'some', 'down', 'under', 'her', 'weren', 'once', 'yourselves', 'youd', 'myself', 'didnt', 'thi', 'the', 'dont', 'but', 'their', 'werent', 'yourself', 'over', 'mightnt', 'hes', 'too', 'each', 'yal', 'when', 'that', 'hey', 'everything', 'with', 'out', 'who', 'both', 'few', 'else', 'than', 'mightn', 'own', 'haven', 'everyone', 'then', 'our', 'was', 'themselves', 'isnt', 'gues', 'arent', 'where', 'youll', 'how', 'again', 'hasnt', 'another', 'why', 'more', 'hadnt', 'shant', 'doing', 'having', 'while', 'were', 'youve', 'been', 'she', 'and', 'there', 'needn', 'hadn', 'wasnt', 'against', 'him', 'wont', 'cant', 'these', 'wasn', 'himself', 'theirs', 'same', 'because', 'wouldn', 'your', 'whom', 'until', 'does', 'here', 'video', 'aren', 'just', 'thatll', 'mustnt', 'nor', 'shouldn', 'wouldnt','would', 'will', 'them', 'above', 'his', 'youre', 'couldnt', 'has', 'wil', 'shan', 'isn', 'herself', 'ain', 'anyone', 'this', 'about', 'after', 'shouldve', 'further', 'every', 'from', 'what', 'shes', 'shouldnt', 'during', 'something', 'between', 'other', 'yours', 'though', 'ours', 'had', 'also', 'should', 'for', 'those', 'they', 'not', 'most', 'off', 'through', 'being', 'did', 'didn', 'can', 'ourselves', 'havent', 'won', 'very', 'are']


In [2]:
#Creating DataFrames
df_comments_us=pd.read_csv('/kaggle/input/youtube/UScomments.csv',on_bad_lines='skip')
df_comments_gb=pd.read_csv('/kaggle/input/youtube/GBcomments.csv',on_bad_lines='skip')
df_videos_us=pd.read_csv('/kaggle/input/youtube/USvideos.csv',on_bad_lines='skip')
df_videos_gb=pd.read_csv('/kaggle/input/youtube/GBvideos.csv',on_bad_lines='skip')

#Connecting GB and US DataFrames
df_videos=pd.concat([df_videos_us, df_videos_gb], ignore_index=True)
df_comments=pd.concat([df_comments_us, df_comments_gb], ignore_index=True)

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
#Creating new DataFrame for normalize data
new_df=pd.DataFrame()

#New DF with videos that have 50+ comments and 1000+ likes. I think less isn't indicative
new_df[['video_id','likes','dislikes']]=df_videos[df_videos['comment_total']>49][df_videos['likes']>999][['video_id','likes','dislikes']]

#New column with likes/dislikes ratio 
new_df['preferences'] = new_df['likes']/(new_df['dislikes']+new_df['likes'])
print(new_df)

          video_id   likes  dislikes  preferences
0      XpVt6Z1Gjjo  320053      5931     0.981806
2      cLdxuaxaQwc  576597     39774     0.935471
3      WYYvHb03Eog   24975      4542     0.846123
4      sjlHnJvXdQs   96666       568     0.994158
5      cMKX2tE5Luk   34507       544     0.984480
...            ...     ...       ...          ...
15979  qnSp6GwsI78   18405       491     0.974016
15980  T2RUYYs8Hxc    1846        92     0.952528
15982  1zOPtQNChZM    6592       127     0.981098
15983  Z7R8XRKqHAI   44811       652     0.985659
15984  lLN1FwiqGwc   58532      3223     0.947810

[13108 rows x 4 columns]


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


In [4]:
#I'd like to create model wich can predict likes ratio 
#But I feel that this dataset is not enough :(
#So my another one model will predict video in top 50% or not 
#So I need to find median and create binary classification for videos 

#Find median
median=new_df['preferences'].median() 
print(f'Median is {round(median,5)}. It means 50% of videos have likes ratio above/below median')

#Create new column with binray preferences 
new_df['prefers_binary']=new_df['preferences'].apply(lambda b:1 if b>=median else 0)
print(new_df)

Median is 0.97677. It means 50% of videos have likes ratio above/below median
          video_id   likes  dislikes  preferences  prefers_binary
0      XpVt6Z1Gjjo  320053      5931     0.981806               1
2      cLdxuaxaQwc  576597     39774     0.935471               0
3      WYYvHb03Eog   24975      4542     0.846123               0
4      sjlHnJvXdQs   96666       568     0.994158               1
5      cMKX2tE5Luk   34507       544     0.984480               1
...            ...     ...       ...          ...             ...
15979  qnSp6GwsI78   18405       491     0.974016               0
15980  T2RUYYs8Hxc    1846        92     0.952528               0
15982  1zOPtQNChZM    6592       127     0.981098               1
15983  Z7R8XRKqHAI   44811       652     0.985659               1
15984  lLN1FwiqGwc   58532      3223     0.947810               0

[13108 rows x 5 columns]


In [5]:
#Now we can prepare text for normalization 
#I'll find all words in comments in each video, fix words and count them
#And in this cell all functions for that
#Some functions are active just in this ver (remove_duplicates_in_words,delete_s)
#Instead of them in full ver I will be use leming or steming


#Here we lost duplicates also in right words but I hope it isn't destroy our model 
def remove_duplicates_in_words(word): #Dooooon't tyyyype coooommmmments liiike thaaaaat =>Don't type coments like that
    l = ''
    new_word = ''
    for i in word:
        if i != l:
            new_word += i
        l = i
    return new_word


#Remove s or ed on end of words
def delete_s_ed(word):
    if word[-1:]=='s':
        new_word=word[:-1]
    elif word[-2:]=='ed':
        new_word=word[:-2]
    else:
        new_word=word
    return new_word


#Only text and text to list of words
def cleaning_text(text): 
    try:
        text=text.lower()
        text=''.join([x for x in text if x in string.ascii_letters + ' ']) #just letters and sapces
    except: #Sometimes no words in comments. People like smiles (:
        text=" "
    return text.split(" ")


#This function remove useless words and fix useful if it needs
#It is light ver so here are not leming or steming
def words_cleaning(words): 
    new_words=[]   
    for word in words:
        word=remove_duplicates_in_words(word) 
        if len(word)<13 and len(word)>=3: # In comments words with length more than 12 are rare and less than 3 are useless
                if word not in stop_words: #Is word in minus(stop) words?
                    new_words.append(delete_s_ed(word))       
    return new_words


#From list of words create dict={word:frequencies} . 
def text_to_frequencies(text): 
    words=set(text)
    frequencies={}
    for word in words:
        frequency=text.count(word)
        frequencies.update({word:frequency})
    return frequencies

In [6]:
#Creating new column with lists of words
n=0
start_time=time()#timer

df_len=len(new_df)
all_comments_words=[]

#Finding comments for each video
for video in new_df['video_id'].values: 
    comments_words=[]
    
    #timer
    n+=1
    if n%500==0:
        print(f"Working, there are still {df_len-n} interation of {df_len}\nFinish in {round((df_len-n)/60*((time()-start_time)/n))} min.")
    #timer
    
    for comment in df_comments[df_comments.video_id==video]['comment_text'].values:
        words=words_cleaning(cleaning_text(comment)) 
        comments_words+=words
    comments_words=text_to_frequencies(comments_words)
    all_comments_words.append(comments_words)

#New column with dict:{word:frequencies} 
new_df['words']=all_comments_words

Working, there are still 12608 interation of 13108
Finish in 44 min.
Working, there are still 12108 interation of 13108
Finish in 43 min.
Working, there are still 11608 interation of 13108
Finish in 40 min.
Working, there are still 11108 interation of 13108
Finish in 38 min.
Working, there are still 10608 interation of 13108
Finish in 36 min.
Working, there are still 10108 interation of 13108
Finish in 34 min.
Working, there are still 9608 interation of 13108
Finish in 33 min.
Working, there are still 9108 interation of 13108
Finish in 31 min.
Working, there are still 8608 interation of 13108
Finish in 30 min.
Working, there are still 8108 interation of 13108
Finish in 28 min.
Working, there are still 7608 interation of 13108
Finish in 26 min.
Working, there are still 7108 interation of 13108
Finish in 25 min.
Working, there are still 6608 interation of 13108
Finish in 22 min.
Working, there are still 6108 interation of 13108
Finish in 21 min.
Working, there are still 5608 interation o

In [7]:
#Find most popular words, create columns for each populae word and add them frequencies in cells
big_dict=Counter() #Dictionary for all words

#Summary all dicts from df.words
for words in new_df['words'].values:
    big_dict.update(words)

#Most popular num_words. Class Counter to dict, dict to items, sort it, generate list of words and slice
new_columns=[k[0] for k in sorted(dict(big_dict).items(), key=lambda item: item[1],reverse=True)][:num_words]

#Creating new columns of most popular words with absolute values of frequencies
for column in new_columns: 
    columns_values=[]
    for words in new_df['words'].values:
        try:
            q=words[column]
        except:
            q=0
        columns_values.append(q)
    new_df[column]=columns_values
    
#Save data. Save time for next time 
new_df.to_csv(f'./data{num_words}_light.csv')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
